In [ ]:
!pip install python-dotenv
!pip install huggingface_hub

In [ ]:
from huggingface_hub import login

login()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Load the Finetuned Model and Tokenizer from HuggingFace

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

username = "cherrymaecaracas"
model_id = f"{username}/gemma-2b-it-ft-data-science-qa"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Test the finetuned model

In [ ]:
def get_completion(question: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """

  <start_of_turn>user
  {question}
  <end_of_turn>

  <start_of_turn>model

  """

  prompt = prompt_template.format(question=question)

  inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
  outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=150)
  result = tokenizer.decode(outputs[0])

  # encoder = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  # model_inputs = encoder.to(device)

  # generated_ids = model.generate(**model_inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
  # generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

  return result

In [ ]:
question = """ What are neural networks? """

result = get_completion(question = question, model = model, tokenizer = tokenizer)
print(result)



  <start_of_turn>user
   What are neural networks? 
  <end_of_turn>

  <start_of_turn>model

  Neural networks are complex systems of interconnected neurons that learn to perform specific tasks. They are used in various applications, including image recognition, speech recognition, and neural networks. <end_of_turn>model Neural networks are complex systems of interconnected neurons that learn to perform specific tasks. They are used in various applications, including image recognition, speech recognition, and neural networks. <end_of_turn>model Neural networks are complex systems of interconnected neurons that learn to perform specific tasks. They are used in various applications, including image recognition, speech recognition, and neural networks. <end_of_turn>model Neural networks are complex systems of interconnected neurons that learn to perform specific tasks. They are used in various applications, including image recognition, speech recognition, and neural networks. <end_of_tu

In [ ]:
result.split("<start_of_turn>model")[1].split("<end_of_turn>")[0]

'\n\n  Neural networks are complex systems of interconnected neurons that learn to perform specific tasks. They are used in various applications, including image recognition, speech recognition, and neural networks. '